In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from time import sleep
import datetime
import bs4
import json 


def clean_string(column):
    return column.apply(lambda x: x.replace("\n",'',2)).apply(lambda x: x.replace('  ',''))

def scrape_reviews(PATH, n_pages, sleep_time = 0.3):


    names = []
    ratings = []
    headers = []
    reviews = []
    dates = []
    locations = []

    for p in range(18):

        sleep(sleep_time)

        #http = requests.get(f'{PATH}{p}&stars=1&stars=5')
        x= PATH+str(p+1)
       
        r = requests.get(x)
         
        bsoup = BeautifulSoup(r.text, 'html.parser')
        base_url = 'https://fr.trustpilot.com/review/mcdonalds.fr'

        review_containers = bsoup.find_all('div', class_ = 'review-content__body')
        user_containers = bsoup.find_all('div', class_ = 'consumer-information__name')
        rating_container = bsoup.find_all('div', class_="star-rating star-rating--medium")      
       
        profile_link_containers = bsoup.find_all('aside', class_ = 'review__consumer-information' )
        
        for x in range(len(bsoup.find_all('div', class_ = 'review-content'))):
            
            review_c = review_containers[x]
            headers.append(review_c.h2.a.text)
            
            if type(review_c.p) == bs4.element.Tag:
                reviews.append(review_c.p.text)
            else :
                reviews.append("")
            reviewer = user_containers[x]
          
            rating = rating_container[x]
            ratings.append( "\n".join([img['alt'] for img in rating.find_all('img', alt=True)]).replace(" étoile :","").replace(" étoiles :",""))
            
           
            
            prof = profile_link_containers[x]
            link = 'https://www.trustpilot.com'+ prof.a['href']
            c_profile = requests.get(f'{link}')
            csoup = BeautifulSoup(c_profile.text, 'html.parser')
            cust_container = csoup.find('div', class_ = 'user-summary-location')
            locations.append(cust_container.text)
    
        for star in  bsoup.find_all('div', class_ = 'review-content'):
          
        # Get date value
          date_json = json.loads(star.find('script').text)
          date = date_json['publishedDate']
          dates.append(date)

    rev_df = pd.DataFrame(list(zip( headers, reviews, ratings, ratings, dates, locations)),
                  columns = ['Header','Review','Rating','Mention','Date', 'Location'])
    
    print(len(bsoup.find_all('div', class_ = 'review-content')))
    
    rev_df.Review = clean_string(rev_df.Review)
    rev_df.Location = clean_string(rev_df.Location)
    rev_df.Location = rev_df.Location.apply(lambda x: x.split(',',1)[-1])
    rev_df.Rating = rev_df.Rating.str[:1]  
    rev_df.Mention = rev_df.Mention.str[2:] 
    rev_df.Date = rev_df.Date.str[:10] 
    return rev_df

In [ ]:
df = scrape_reviews(PATH = 'https://fr.trustpilot.com/review/mcdonalds.fr?page=',
                   n_pages = 18)

11


In [ ]:
df

,Header,Review,Rating,Mention,Date,Location
0,Est ce que le covid empeche les…,Est ce que le covid empeche les employes de di...,1,mauvais,2020-05-18,France
1,Macdo Cesson personnel incompétent,Bonjour je reviens du Macdonald de boisenart à...,1,mauvais,2020-05-17,France
2,De plus en plus petit les portions depuis le c...,De plus en plus petit depuis le confinement..m...,1,mauvais,2020-05-16,France
3,Burger sans saveur sans salade sans…,Burger sans saveur sans salade sans tomates en...,1,mauvais,2020-05-15,France
4,Macdonald’s plaisance du touch 31830,Macdonald’s plaisance du touch 31830 : NUL !!!...,1,mauvais,2020-05-15,France
...,...,...,...,...,...,...
346,Mon fast-food préféré,"J'aime beaucoup aller chez McDonald, surtout q...",4,bien,2016-03-28,France
347,Degueulasse,"Dégueulasse ! Frites immonde, service lent.Le ...",1,mauvais,2015-09-08,France
348,genial a part les sodas,c est sympa surtout pour que les enfants puiss...,5,excellent,2015-07-03,Malta
349,pas toujours de glace,"Effectivement, rarement de la glace le soir..",4,bien,2014-07-25,France


In [ ]:
df.write.format("csv").save("//Desktop")


AttributeError: ignored

SyntaxError: ignored

In [ ]:
import sys
# !{sys.executable} -m spacy download en
import re, numpy as np, pandas as pd
from pprint import pprint

# Gensim
import gensim
import spacy
import logging
import warnings
import gensim.corpora as corpora
from gensim.utils import lemmatize, simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt
import gensim.models 

# NLTK Stop words
from nltk.corpus import stopwords
#from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop 
stop_words = stopwords.words('french')
stop_words.extend(['ca', 'ete','donc','apres','tout','cest','quand','très','ça','daller','plus','do','fois','plus','car','tous','alors','tres','lors','faire','sans','dautre','nest','encore','comment','choose','prendre','arrive','meme','avant','etaient','ver','meet','toute','vraiment','toujour','tient','dire','quelque','fait','renverse'])
%matplotlib inline
warnings.filterwarnings("ignore",category=DeprecationWarning)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

LookupError: ignored

In [ ]:
import nltk
nltk.download('all')

In [ ]:
def sent_to_words(sentences):
    for sent in sentences:
        sent = re.sub('\S*@\S*\s?', '', sent)  # remove emails
        sent = re.sub('\s+', ' ', sent)  # remove newline chars
        sent = re.sub("\'", "", sent)  # remove single quotes
        sent = gensim.utils.simple_preprocess(str(sent), deacc=True) 
        yield(sent)  

# Convert to list
data = df.Review.values.tolist()
data_words = list(sent_to_words(data))
print(data_words[:1])

In [ ]:

!python3 -m spacy download fr_core_news_sm

In [ ]:
import fr_core_news_sm
nlp = fr_core_news_sm.load()


In [ ]:
# Build the bigram and trigram models
from nltk.stem.snowball import FrenchStemmer
Stemmer = FrenchStemmer()
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# !python3 -m spacy download en  # run in terminal once
# or do
# !conda install -c conda-forge spacy-model-en_core_web_md 
# and use nlp=spacy.load('en_core_web_sm') instead in below function.
def process_words(texts, stop_words=stop_words, allowed_postags=['NOUN', 'ADJ', 'VERB']):
    """Remove Stopwords, Form Bigrams, Trigrams and Lemmatization"""
    texts = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
    texts = [bigram_mod[doc] for doc in texts]
    texts = [trigram_mod[bigram_mod[doc]] for doc in texts]
    texts_out = []
    nlp = fr_core_news_sm.load()
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([Stemmer.stem(token.text) for token in doc if token.pos_ in allowed_postags])
    # remove stopwords once more after lemmatization
    texts_out = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts_out]    
    return texts_out


data_ready = process_words(data_words)  # processed Text Data!

In [ ]:
print(data_ready)

In [ ]:
# Load the library with the CountVectorizer method
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

def plot_10_most_common_words(count_data, count_vectorizer):
    import matplotlib.pyplot as plt
    words = count_vectorizer.get_feature_names()
    total_counts = np.zeros(len(words))
    for t in count_data:
        total_counts+=t.toarray()[0]
    
    count_dict = (zip(words, total_counts))
    count_dict = sorted(count_dict, key=lambda x:x[1], reverse=True)[0:10]
    words = [w[0] for w in count_dict]
    counts = [w[1] for w in count_dict]
    x_pos = np.arange(len(words)) 
    
    plt.figure(2, figsize=(15, 15/1.6180))
    plt.subplot(title='10 most common words')
    sns.set_context("notebook", font_scale=1.25, rc={"lines.linewidth": 2.5})
    sns.barplot(x_pos, counts, palette='husl')
    plt.xticks(x_pos, words, rotation=90) 
    plt.xlabel('words')
    plt.ylabel('counts')
    plt.show()
# Initialise the count vectorizer with the English stop words
count_vectorizer = CountVectorizer(stop_words=stop_words )
# Fit and transform the processed titles
count_data = count_vectorizer.fit_transform(data)
# Visualise the 10 most common words
plot_10_most_common_words(count_data, count_vectorizer)




In [ ]:
# Create Dictionary
from gensim import corpora, models
id2word = corpora.Dictionary(data_ready)
id2word.filter_extremes(no_below=7, no_above=0.8, keep_n=1000000)
# Create Corpus: Term Document Frequency
corpus = [id2word.doc2bow(text) for text in data_ready]
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus_tfidf,
                                           id2word=id2word,
                                           num_topics=4, 
                                           random_state=70,
                                           update_every=1,
                                           chunksize=20,
                                           passes=100,
                                           alpha='symmetric',
                                           iterations=100,
                                           per_word_topics=True)

pprint(lda_model.print_topics())

In [ ]:
num_topic=4
def get_lda_topics(model, num_topics):
    word_dict = {};
    for i in range(num_topics):
        words = model.show_topic(i, topn = 20);
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = [i[0] for i in words];
    return pd.DataFrame(word_dict);
get_lda_topics(lda_model, num_topic)


In [ ]:
def format_topics_sentences(ldamodel=None, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic i each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data_ready)
# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
df_dominant_topic.head(10)

In [ ]:
Aspect =pd.DataFrame(pd.cut(df_dominant_topic['Dominant_Topic'],bins=[-1,0,1,2,3],labels=['Restaurant en general ','Service','Commande/drive','Nourriture']))

Aspect.columns=["Aspect"]


In [ ]:
result = pd.concat([df, Aspect], axis=1)
result.head(10)

In [ ]:
# Display setting to show more characters in column
pd.options.display.max_colwidth = 100

sent_topics_sorteddf_mallet = pd.DataFrame()
sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=False).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Representative Text"]

# Show
sent_topics_sorteddf_mallet.head(10)

In [ ]:
# 1. Wordcloud of Top N words in each topic
from matplotlib import pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import matplotlib.colors as mcolors

cols = [color for name, color in mcolors.TABLEAU_COLORS.items()]  # more colors: 'mcolors.XKCD_COLORS'

cloud = WordCloud(stopwords=stop_words,
                  background_color='white',
                  width=2500,
                  height=1800,
                  max_words=10,
                  colormap='tab10',
                  color_func=lambda *args, **kwargs: cols[i],
                  prefer_horizontal=1.0)

topics = lda_model.show_topics(formatted=False)

fig, axes = plt.subplots(2, 2, figsize=(10,10), sharex=True, sharey=True)

for i, ax in enumerate(axes.flatten()):
    fig.add_subplot(ax)
    topic_words = dict(topics[i][1])
    cloud.generate_from_frequencies(topic_words, max_font_size=300)
    plt.gca().imshow(cloud)
    plt.gca().set_title('Topic ' + str(i), fontdict=dict(size=16))
    plt.gca().axis('off')


plt.subplots_adjust(wspace=0, hspace=0)
plt.axis('off')
plt.margins(x=0, y=0)
plt.tight_layout()
plt.show()

In [ ]:
from collections import Counter
topics = lda_model.show_topics(formatted=False)
data_flat = [w for w_list in data_ready for w in w_list]
counter = Counter(data_flat)

out = []
for i, topic in topics:
    for word, weight in topic:
        out.append([word, i , weight, counter[word]])

df = pd.DataFrame(out, columns=['word', 'topic_id', 'importance', 'word_count'])        

# Plot Word Count and Weights of Topic Keywords
fig, axes = plt.subplots(2, 2, figsize=(16,10), sharey=True, dpi=160)
cols = [color for name, color in mcolors.TABLEAU_COLORS.items()]
for i, ax in enumerate(axes.flatten()):
    #ax.bar(x='word', height="word_count", data=df.loc[df.topic_id==i, :], color=cols[i], width=0.5, alpha=0.3, label='Word Count')
    ax_twin = ax.twinx()
    ax_twin.bar(x='word', height="importance", data=df.loc[df.topic_id==i, :], color=cols[i], width=0.2, label='Weights')
    #ax.set_ylabel('Word Count', color=cols[i])
    ax_twin.set_ylim(0, 0.13); 
    ax.set_title('Topic: ' + str(i), color=cols[i], fontsize=16)
    ax.tick_params(axis='y', left=False)
    ax.set_xticklabels(df.loc[df.topic_id==i, 'word'], rotation=30, horizontalalignment= 'right')
    ax.legend(loc='upper left'); ax_twin.legend(loc='upper right')

fig.tight_layout(w_pad=2)    
fig.suptitle(' Importance of Topic Keywords', fontsize=22, y=1.05)    
plt.show()

In [ ]:
from collections import Counter
topics = lda_model.show_topics(formatted=False)
data_flat = [w for w_list in data_ready for w in w_list]
counter = Counter(data_flat)

out = []
for i, topic in topics:
    for word, weight in topic:
        out.append([word, i , weight, counter[word]])

df = pd.DataFrame(out, columns=['word', 'topic_id', 'importance', 'word_count'])        

# Plot Word Count and Weights of Topic Keywords
fig, axes = plt.subplots(2, 2, figsize=(16,10), sharey=True, dpi=160)
cols = [color for name, color in mcolors.TABLEAU_COLORS.items()]
for i, ax in enumerate(axes.flatten()):
    ax.bar(x='word', height="word_count", data=df.loc[df.topic_id==i, :], color=cols[i], width=0.5, alpha=0.3, label='Word Count')
    ax_twin = ax.twinx()
    #ax_twin.bar(x='word', height="importance", data=df.loc[df.topic_id==i, :], color=cols[i], width=0.2, label='Weights')
    ax.set_ylabel('Word Count', color=cols[i])
    #ax_twin.set_ylim(0, 0.030); ax.set_ylim(0, 3500)
    ax.set_title('Topic: ' + str(i), color=cols[i], fontsize=16)
    ax.tick_params(axis='y', left=False)
    ax.set_xticklabels(df.loc[df.topic_id==i, 'word'], rotation=30, horizontalalignment= 'right')
    ax.legend(loc='upper left'); ax_twin.legend(loc='upper right')

fig.tight_layout(w_pad=2)    
fig.suptitle('Word Count for each topic', fontsize=22, y=1.05)    
plt.show()

In [ ]:
# Sentence Coloring of N Sentences
from matplotlib.patches import Rectangle

def sentences_chart(lda_model=lda_model, corpus=corpus_tfidf, start = 70, end = 77):
    corp = corpus[start:end]
    mycolors = [color for name, color in mcolors.TABLEAU_COLORS.items()]

    fig, axes = plt.subplots(end-start, 1, figsize=(20, (end-start)*0.95), dpi=160)       
    axes[0].axis('off')
    for i, ax in enumerate(axes):
        if i > 0:
            corp_cur = corp[i-1] 
            topic_percs, wordid_topics, wordid_phivalues = lda_model[corp_cur]
            word_dominanttopic = [(lda_model.id2word[wd], topic[0]) for wd, topic in wordid_topics]    
            ax.text(0.01, 0.5, "Doc " + str(i-1) + ": ", verticalalignment='center',
                    fontsize=16, color='black', transform=ax.transAxes, fontweight=700)

            # Draw Rectange
            topic_percs_sorted = sorted(topic_percs, key=lambda x: (x[1]), reverse=True)
            ax.add_patch(Rectangle((0.0, 0.05), 0.99, 0.90, fill=None, alpha=1, 
                                   color=mycolors[topic_percs_sorted[0][0]], linewidth=2))

            word_pos = 0.06
            for j, (word, topics) in enumerate(word_dominanttopic):
                if j < 14:
                    ax.text(word_pos, 0.5, word,
                            horizontalalignment='left',
                            verticalalignment='center',
                            fontsize=16, color=mycolors[topics],
                            transform=ax.transAxes, fontweight=700)
                    word_pos += .009 * len(word)  # to move the word for the next iter
                    ax.axis('off')
            ax.text(word_pos, 0.5, '. . .',
                    horizontalalignment='left',
                    verticalalignment='center',
                    fontsize=16, color='black',
                    transform=ax.transAxes)       

    plt.subplots_adjust(wspace=0, hspace=0)
    plt.suptitle('Sentence Topic Coloring for Documents: ' + str(start) + ' to ' + str(end-2), fontsize=22, y=0.95, fontweight=700)
    plt.tight_layout()
    plt.show()

sentences_chart()    

In [ ]:
# Sentence Coloring of N Sentences
def topics_per_document(model, corpus, start=0, end=1):
    corpus_sel = corpus[start:end]
    dominant_topics = []
    topic_percentages = []
    for i, corp in enumerate(corpus_sel):
        topic_percs, wordid_topics, wordid_phivalues = model[corp]
        dominant_topic = sorted(topic_percs, key = lambda x: x[1], reverse=True)[0][0]
        dominant_topics.append((i, dominant_topic))
        topic_percentages.append(topic_percs)
    return(dominant_topics, topic_percentages)

dominant_topics, topic_percentages = topics_per_document(model=lda_model, corpus=corpus, end=-1)            

# Distribution of Dominant Topics in Each Document
df = pd.DataFrame(dominant_topics, columns=['Document_Id', 'Dominant_Topic'])
dominant_topic_in_each_doc = df.groupby('Dominant_Topic').size()
df_dominant_topic_in_each_doc = dominant_topic_in_each_doc.to_frame(name='count').reset_index()

# Total Topic Distribution by actual weight
topic_weightage_by_doc = pd.DataFrame([dict(t) for t in topic_percentages])
df_topic_weightage_by_doc = topic_weightage_by_doc.sum().to_frame(name='count').reset_index()

# Top 3 Keywords for each Topic
topic_top3words = [(i, topic) for i, topics in lda_model.show_topics(formatted=False) 
                                 for j, (topic, wt) in enumerate(topics) if j < 3]

df_top3words_stacked = pd.DataFrame(topic_top3words, columns=['topic_id', 'words'])
df_top3words = df_top3words_stacked.groupby('topic_id').agg(', \n'.join)
df_top3words.reset_index(level=0,inplace=True)

In [ ]:
from matplotlib.ticker import FuncFormatter

# Plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4), dpi=120, sharey=True)

# Topic Distribution by Dominant Topics
ax1.bar(x='Dominant_Topic', height='count', data=df_dominant_topic_in_each_doc, width=.5, color='firebrick')
ax1.set_xticks(range(df_dominant_topic_in_each_doc.Dominant_Topic.unique().__len__()))
tick_formatter = FuncFormatter(lambda x, pos: 'Topic ' + str(x)+ '\n' + df_top3words.loc[df_top3words.topic_id==x, 'words'].values[0])
ax1.xaxis.set_major_formatter(tick_formatter)
ax1.set_title('Number of Documents by Dominant Topic', fontdict=dict(size=10))
ax1.set_ylabel('Number of Documents')
ax1.set_ylim(0, 200)

# Topic Distribution by Topic Weights
ax2.bar(x='index', height='count', data=df_topic_weightage_by_doc, width=.5, color='steelblue')
ax2.set_xticks(range(df_topic_weightage_by_doc.index.unique().__len__()))
ax2.xaxis.set_major_formatter(tick_formatter)
ax2.set_title('Number of Documents by Topic Weightage', fontdict=dict(size=10))

plt.show()

In [ ]:
# Number of Documents for Each Topic
topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()

# Percentage of Documents for Each Topic
topic_contribution = round(topic_counts/topic_counts.sum(), 4)

# Topic Number and Keywords
topic_num_keywords = df_topic_sents_keywords[['Dominant_Topic', 'Topic_Keywords']]

# Concatenate Column wise
df_dominant_topics = pd.concat([topic_num_keywords, topic_counts, topic_contribution], axis=1)

# Change Column names
df_dominant_topics.columns = ['Dominant_Topic', 'Topic_Keywords', 'Num_Documents', 'Perc_Documents']

# Show
df_dominant_topics[:3]

In [ ]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word)
vis


In [ ]:
!pip install pyLDAvis

##  Topic modeling with NMF

In [ ]:
import pandas as pd;
import numpy as np;
import scipy as sp;
import sklearn;
import sys;
from nltk.corpus import stopwords;
import nltk;
from gensim.models import ldamodel
import gensim.corpora;
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer;
from sklearn.decomposition import NMF;
from sklearn.preprocessing import normalize;
import pickle;

In [ ]:
train_headlines_sentences = [' '.join(text) for text in data_ready]

In [ ]:
vectorizer = CountVectorizer(analyzer='word', max_features=5000);
x_counts = vectorizer.fit_transform(train_headlines_sentences);

In [ ]:
transformer = TfidfTransformer(smooth_idf=False);
x_tfidf = transformer.fit_transform(x_counts);

In [ ]:
xtfidf_norm = normalize(x_tfidf, norm='l1', axis=1)


In [ ]:
#obtain a NMF model.
model = NMF(n_components=4, init='nndsvd',shuffle=True);
#fit the model
model.fit(xtfidf_norm)

In [ ]:
def get_nmf_topics(model, n_top_words):
    
    #the word ids obtained need to be reverse-mapped to the words so we can print the topic names.
    feat_names = vectorizer.get_feature_names()
    
    word_dict = {};
    for i in range(4):
        
        #for each topic, obtain the largest values, and add the words they map to into the dictionary.
        words_ids = model.components_[i].argsort()[:-10 - 1:-1]
        words = [feat_names[key] for key in words_ids]
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = words;
    
    return pd.DataFrame(word_dict);

In [ ]:
get_nmf_topics(model,10)

In [ ]:
####################################### NEXT STEP OF OPINION MINING ###########################################

In [ ]:
result.head(10)

In [ ]:
import matplotlib.pyplot as plt

plot_size = plt.rcParams["figure.figsize"] 
print(plot_size[0]) 
print(plot_size[1])

plot_size[0] = 10
plot_size[1] = 10
plt.rcParams["figure.figsize"] = plot_size 
result.Mention.value_counts().plot(kind='pie', autopct='%1.0f%%')

In [ ]:

list2=[]

for line in result['Rating']:   
    if line in ["1","2"]:
      list2.append("negative")
    elif line =="3":
      list2.append("neutre")
    else :
       list2.append("positive")

            


In [ ]:
!pip install vaderSentiment

In [ ]:
sentiment= pd.DataFrame(list2)
sentiment.columns=["sentiment"]
resultFinal = pd.concat([result, sentiment], axis=1)
resultFinal.head(10)

In [ ]:
print(sentiment.groupby('sentiment').size().sort_values(ascending=False))
resultFinal.sentiment.value_counts().plot(kind='pie', autopct='%1.0f%%')

In [ ]:
resultFinal.Aspect.value_counts().plot(kind='pie', autopct='%1.0f%%')

In [ ]:
# reviews_train.columns
print(resultFinal.groupby('Aspect').size().sort_values(ascending=False))

#how many categories
print("number of categories",resultFinal.Aspect.nunique())

In [ ]:
resultFinalWork=resultFinal


In [ ]:
!python3 -m spacy download fr_core_news_md

### ***TRY AMA BAED BICH NAHEHA KEN MAMCHETICH ***



In [ ]:
import spacy
import fr_core_news_md
nlp = fr_core_news_md.load()

resultFinalWork.Review = resultFinalWork.Review.str.lower()

aspect_terms = []
for review in nlp.pipe(resultFinalWork.Review):
    chunks = [(chunk.root.text) for chunk in review.noun_chunks if chunk.root.pos_ == 'NOUN']
    aspect_terms.append(' '.join(chunks))
resultFinalWork['aspect_terms'] = aspect_terms
resultFinalWork.head(10)

In [ ]:
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Dense, Activation

aspect_categories_model = Sequential()
aspect_categories_model.add(Dense(512, input_shape=(6000,), activation='relu'))
aspect_categories_model.add(Dense(4, activation='softmax'))
aspect_categories_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
from keras.preprocessing.text import Tokenizer

vocab_size = 6000 # We set a maximum size for the vocabulary
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(resultFinalWork.Review)
aspect_tokenized = pd.DataFrame(tokenizer.texts_to_matrix(resultFinalWork.aspect_terms))

In [ ]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

label_encoder = LabelEncoder()
integer_category = label_encoder.fit_transform(resultFinalWork.Aspect)
dummy_category = to_categorical(integer_category)

In [ ]:
aspect_categories_model.fit(aspect_tokenized, dummy_category, epochs=6, verbose=1)


In [ ]:
new_review = "le service est horrible "

chunks = [(chunk.root.text) for chunk in nlp(new_review).noun_chunks if chunk.root.pos_ == 'NOUN']
new_review_aspect_terms = ' '.join(chunks)
new_review_aspect_tokenized = tokenizer.texts_to_matrix([new_review_aspect_terms])

new_review_category = label_encoder.inverse_transform(aspect_categories_model.predict_classes(new_review_aspect_tokenized))
print(new_review_category)

In [ ]:
sentiment_terms = []
for review in nlp.pipe(resultFinalWork['Review']):
        if review.is_parsed:
            sentiment_terms.append(' '.join([token.text for token in review if (not token.is_stop and not token.is_punct and (token.pos_ == "ADJ" or token.pos_ == "VERB"))]))
        else:
            sentiment_terms.append('')  
resultFinalWork['sentiment_terms'] = sentiment_terms
resultFinalWork.head(10)

In [ ]:
resultFinalWork.head()

NameError: ignored

In [ ]:
Trutpilot_sentiment = resultFinalWork.groupby(['Aspect', 'sentiment']).sentiment.count().unstack()
Trutpilot_sentiment.plot(kind='bar')

In [ ]:
sentiment_model = Sequential()
sentiment_model.add(Dense(512, input_shape=(6000,), activation='relu'))
sentiment_model.add(Dense(218, input_shape=(6000,), activation='relu'))
sentiment_model.add(Dense(64, input_shape=(6000,), activation='relu'))
sentiment_model.add(Dense(3, activation='softmax'))
sentiment_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
sentiment_tokenized = pd.DataFrame(tokenizer.texts_to_matrix(resultFinalWork.sentiment_terms))

In [ ]:
label_encoder_2 = LabelEncoder()
integer_sentiment = label_encoder_2.fit_transform(resultFinalWork.sentiment)
dummy_sentiment = to_categorical(integer_sentiment)

In [ ]:
sentiment_model.fit(sentiment_tokenized, dummy_sentiment, epochs=5, verbose=1)

In [ ]:
new_review = "mauvais service"

chunks = [(chunk.root.text) for chunk in nlp(new_review).noun_chunks if chunk.root.pos_ == 'NOUN']
new_review_aspect_terms = ' '.join(chunks)
new_review_aspect_tokenized = tokenizer.texts_to_matrix([new_review_aspect_terms])

new_review_category = label_encoder_2.inverse_transform(sentiment_model.predict_classes(new_review_aspect_tokenized))
print(new_review_category)


In [ ]:
test_reviews = [
    "Bon service rapide.",
    "L'hôtesse était très desagréable.",
    "Le pain était rassis, la salade était trop chère et vide.",
    "La nourriture que nous avons commandée était mauvais, même si je ne dirais pas que les margaritas étaient quelque chose à raconter.",
    "Cet endroit a un décor totalement bizarre, des escaliers avec des murs en miroir - je suis surpris de voir que personne ne s'est encore cassé la tête ou n'est tombé des escaliers"
]

# Aspect preprocessing
test_reviews = [review.lower() for review in test_reviews]
test_aspect_terms = []
for review in nlp.pipe(test_reviews):
    chunks = [(chunk.root.text) for chunk in review.noun_chunks if chunk.root.pos_ == 'NOUN']
    test_aspect_terms.append(' '.join(chunks))
test_aspect_terms = pd.DataFrame(tokenizer.texts_to_matrix(test_aspect_terms))
                             
# Sentiment preprocessing
test_sentiment_terms = []
for review in nlp.pipe(test_reviews):
        if review.is_parsed:
            test_sentiment_terms.append(' '.join([token.lemma_ for token in review if (not token.is_stop and not token.is_punct and (token.pos_ == "ADJ" or token.pos_ == "VERB"))]))
        else:
            test_sentiment_terms.append('') 
test_sentiment_terms = pd.DataFrame(tokenizer.texts_to_matrix(test_sentiment_terms))

# Models output
test_aspect_categories = label_encoder.inverse_transform(aspect_categories_model.predict_classes(test_aspect_terms))
test_sentiment = label_encoder_2.inverse_transform(sentiment_model.predict_classes(test_sentiment_terms))
for i in range(5):
    print("Review " + str(i+1) + " is expressing a  " + test_sentiment[i] + " opinion about " + test_aspect_categories[i])

# ***Analyse du concurant***


